In [1]:
import requests
from datetime import date, timedelta
import pandas as pd
import os
from dotenv import load_dotenv  # pip install python-dotenv

load_dotenv()  # loads .env if present
TOKEN = os.environ["BANXICO_API_TOKEN"]

ID = "SF43718"  # FIX
hoy = date.today()
ini = (hoy - timedelta(days=15)).isoformat()
fin = hoy.isoformat()

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{ID}/datos/{ini}/{fin}"
headers = {"Bmx-Token": TOKEN, "Accept": "application/json"}

r = requests.get(url, headers=headers, timeout=20)
r.raise_for_status()
datos = r.json()["bmx"]["series"][0]["datos"]

# Filtra datos válidos y toma el más reciente
datos_validos = [d for d in datos if d["dato"] not in ("N/E", "", None)]
ultimo = max(datos_validos, key=lambda d: d["fecha"]) if datos_validos else None

print(ultimo)  # {'fecha': '2025-09-15', 'dato': 'XX.XXXX'}

{'fecha': '19/09/2025', 'dato': '18.3892'}


In [5]:
import yfinance as yf
import pandas as pd

# Símbolo de Yahoo Finance para USD/MXN
symbol = "MXN=X"

# Descargamos ~15 días y filtramos los últimos 10 válidos
df = yf.download(symbol, period="15d", interval="1d", progress=False)

# Nos quedamos solo con la columna Close (precio de cierre diario)
df_fx = df[["Close"]].dropna().tail(10).copy()

# Renombramos la columna a algo más claro
df_fx.rename(columns={"Close": "USD_MXN"}, inplace=True)

# Calculamos extra: variación porcentual diaria y media móvil de 7 días
df_fx["ret_diario_%"] = df_fx["USD_MXN"].pct_change().mul(100).round(4)
df_fx["MA_7d"] = df_fx["USD_MXN"].rolling(7).mean().round(6)

# El DataFrame resultante
print(df_fx)

/tmp/ipykernel_7061/2921964509.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="15d", interval="1d", progress=False)


Price         USD_MXN ret_diario_%      MA_7d
Ticker          MXN=X                        
Date                                         
2025-09-09  18.652870          NaN        NaN
2025-09-10  18.623501      -0.1575        NaN
2025-09-11  18.580799      -0.2293        NaN
2025-09-12  18.454000      -0.6824        NaN
2025-09-15  18.446989      -0.0380        NaN
2025-09-16  18.361521      -0.4633        NaN
2025-09-17  18.285801      -0.4124  18.486497
2025-09-18  18.307800       0.1203  18.437202
2025-09-19  18.364201       0.3081  18.400159
2025-09-20  18.322500      -0.2271  18.363259


In [7]:
import requests
from datetime import date, timedelta
import pandas as pd
import yfinance as yf

# =====================
# 1) Banxico (SIE-API)
# =====================
TOKEN = os.environ["BANXICO_API_TOKEN"]
ID = "SF43718"               # FIX (USD/MXN)
hoy = date.today()
ini = (hoy - timedelta(days=15)).isoformat()
fin = hoy.isoformat()

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{ID}/datos/{ini}/{fin}"
headers = {"Bmx-Token": TOKEN, "Accept": "application/json"}

r = requests.get(url, headers=headers, timeout=20)
r.raise_for_status()
serie = r.json()["bmx"]["series"][0]

df_bmx = pd.DataFrame(serie["datos"])
# Limpieza
df_bmx = df_bmx[df_bmx["dato"].notna() & (df_bmx["dato"] != "N/E")].copy()
df_bmx["USD_MXN"] = df_bmx["dato"].astype(float)
df_bmx["fecha"] = pd.to_datetime(df_bmx["fecha"], dayfirst=True)
df_bmx = df_bmx.sort_values("fecha").set_index("fecha")[["USD_MXN"]]

# Métricas
df_bmx["ret_diario_%"] = df_bmx["USD_MXN"].pct_change().mul(100).round(4)
df_bmx["MA_7d"] = df_bmx["USD_MXN"].rolling(7).mean().round(6)
df_bmx["fuente"] = "banxico"

# =====================
# 2) Yahoo Finance
# =====================
symbol = "MXN=X"
df_yf_raw = yf.download(symbol, period="15d", interval="1d", progress=False)

# Puede venir multiíndice; “exprímelo” a Serie y normaliza
close = df_yf_raw["Close"].squeeze()
df_yf = pd.DataFrame({"USD_MXN": close}).dropna().copy()
df_yf.index.name = "fecha"

# Métricas
df_yf["ret_diario_%"] = df_yf["USD_MXN"].pct_change().mul(100).round(4)
df_yf["MA_7d"] = df_yf["USD_MXN"].rolling(7).mean().round(6)
df_yf["fuente"] = "yahoo"

# Si quieres exactamente 10 días recientes por fuente:
df_bmx_10 = df_bmx.tail(10).copy()
df_yf_10 = df_yf.tail(10).copy()

# =====================
# 3) Unir (formato largo)
# =====================
cols = ["USD_MXN", "ret_diario_%", "MA_7d", "fuente"]
df_largo = pd.concat([df_bmx_10[cols], df_yf_10[cols]], axis=0).sort_index()

print("\n=== Formato largo (ambas fuentes apiladas) ===")
print(df_largo.tail(15))

# =====================
# 4) (Opcional) Vista ancha
#    - Precio por fuente: columnas USD_MXN_banxico / USD_MXN_yahoo
#    - También puedes pivotear ret/MA si lo necesitas (tendrá MultiIndex)
# =====================
precios_ancho = df_largo.reset_index().pivot(index="fecha", columns="fuente", values="USD_MXN")
precios_ancho = precios_ancho.rename_axis(None, axis=1).rename(columns={"banxico":"USD_MXN_banxico",
                                                                        "yahoo":"USD_MXN_yahoo"})
print("\n=== Vista ancha (precios por fuente) ===")
print(precios_ancho.tail(10))

# Si quisieras pivotear TODO (precio, ret, MA) a columnas por fuente:
todo_ancho = (df_largo.reset_index()
              .melt(id_vars=["fecha","fuente"], var_name="metric", value_name="valor")
              .pivot_table(index="fecha", columns=["metric","fuente"], values="valor", aggfunc="first")
              .sort_index(axis=1))
print("\n=== Vista ancha con métricas (MultiIndex) ===")
print(todo_ancho.tail(10))


/tmp/ipykernel_7061/3972376983.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_yf_raw = yf.download(symbol, period="15d", interval="1d", progress=False)



=== Formato largo (ambas fuentes apiladas) ===
              USD_MXN  ret_diario_%      MA_7d   fuente
fecha                                                  
2025-09-10  18.623501       -0.1575  18.687419    yahoo
2025-09-11  18.528700       -0.3405        NaN  banxico
2025-09-11  18.580799       -0.2293  18.677804    yahoo
2025-09-12  18.475700       -0.2860        NaN  banxico
2025-09-12  18.454000       -0.6824  18.638562    yahoo
2025-09-15  18.363500       -0.6073  18.561343  banxico
2025-09-15  18.446989       -0.0380  18.601474    yahoo
2025-09-16  18.361521       -0.4633  18.549434    yahoo
2025-09-17  18.285801       -0.4124  18.486497    yahoo
2025-09-17  18.325700       -0.2058  18.510843  banxico
2025-09-18  18.307800        0.1203  18.437202    yahoo
2025-09-18  18.361000        0.1926  18.468843  banxico
2025-09-19  18.389200        0.1536  18.433686  banxico
2025-09-19  18.364201        0.3081  18.400159    yahoo
2025-09-20  18.322500       -0.2271  18.363259    yahoo


In [ ]:
Data_Science_2025
Repository that stores all the code related to the Data Science Course by DECI UNAM; https://www.deci-unam.com/datos